In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
# Config
import copy
import os
import numpy as np
from numpy.linalg import norm
import matplotlib
import matplotlib.pyplot as plt

# Path for ffmpeg (if animations are needed)
plt.rcParams['animation.ffmpeg_path'] = "C:\\Users\\Fredric\\Documents\\ffmpeg\\ffmpeg-n4.4-latest-win64-gpl-4.4\\bin\\ffmpeg.exe"

# Path for .pkl files output
target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\tungurahua\\pilot_data_upto_r2\\"
# target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\test_oscillations\\"
# target_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\scenarios\\results_dx4m\\"

# Path for Quail source code
source_dir = "C:\\Users\\Fredric\\Documents\\Volcano\\quail_volcano\\src\\"

# Path for Quail entry point
quail_path = os.path.join(source_dir, "quail")

# Name of file to run (must be located in target_dir)
target_file = "conduit_p.py"

In [4]:
from types import SimpleNamespace

In [16]:
# Import quail modules
os.chdir(source_dir)
import argparse
import importlib
import sys
import scipy

import defaultparams as default_deck
import errors
from general import ShapeType, SolverType, PhysicsType

import meshing.common as mesh_common
import meshing.gmsh as mesh_gmsh
import meshing.tools as mesh_tools

import numerics.helpers.helpers as helpers
import numerics.timestepping.tools as stepper_tools

import physics.zerodimensional.zerodimensional as zerod
import physics.euler.euler as euler
import physics.navierstokes.navierstokes as navierstokes
import physics.scalar.scalar as scalar
import physics.chemistry.chemistry as chemistry
import physics.multiphasevpT.multiphasevpT as multiphasevpT

import processing.readwritedatafiles as readwritedatafiles
import processing.post as post
import processing.plot as plot
import processing
# import processing.mdtools as mdtools

import solver.DG as DG
import solver.ADERDG as ADERDG
import solver.tools as solver_tools

import time
import multiprocessing as mp
from multidomain import Domain, Observer

os.chdir(target_dir)

In [5]:
implicitdiff = SimpleNamespace()
implicitdiff.c = np.zeros(2)
implicitdiff.cspeed = 0.
implicitdiff.al = np.zeros(2)

In [18]:
''' Attach some Quail output for suitable data input shapes. '''
solver1D_par = lambda solver_idx, i: readwritedatafiles.read_data_file(
  #f"featuretest_conduit1_{i}.pkl")
  f"tung_atm{solver_idx}_{i}.pkl")
solver = solver1D_par(1,0)

In [8]:
A = np.zeros((3,4,5))
B = np.zeros((3,4,5))
np.stack([A,B],axis=-1).shape

(3, 4, 5, 2)

In [ ]:
class ConstAdvDiffScalar2D(ConstAdvDiffScalar):
	NDIMS = 2

	def set_physical_params(self, ConstXVelocity=1., ConstYVelocity=1., 
			DiffCoefficientX=1., DiffCoefficientY=1.):

		self.c = np.array([ConstXVelocity, ConstYVelocity])
		self.al = np.array([DiffCoefficientX, DiffCoefficientY])
		self.cspeed = np.linalg.norm(self.c)

def get_diff_flux_interior(self, Uq, gUq):
	# Diffusion matrix's diagonal
	al = np.array([1.0, 1.0])
	F = np.empty(Uq.shape + (self.NDIMS,)) # [n, nq, ns, ndims]
	F[:, :, :, 0] = al[0] * gUq[:, :, :, 0]
	F[:, :, :, 1] = al[1] * gUq[:, :, :, 1]

	return F

base_fcns.SIP

In [ ]:
class ConstAdvDiffScalar(base.PhysicsBase):
	'''
	This class corresponds to scalar advection/diffusion with a constant 
	velocity and diffusion coefficient.

	It inherits attributes and methods from the PhysicsBase class. See
	PhysicsBase for detailed comments of attributes and methods. This
	class should not be instantiated directly. Instead, the 1D and 2D
	variants, which inherit from this class (see below), should be
	instantiated.

	Additional methods and attributes are commented below.

	Attributes:
	-----------
	c: float or numpy array
		advection velocity
	al: float or numpy array
		diffusion coefficient
	cspeed: float
		advection speed
	'''
	NUM_STATE_VARS = 1
	PHYSICS_TYPE = general.PhysicsType.ConstAdvDiffScalar

	def __init__(self, mesh):
		super().__init__(mesh)
		self.c = 0.
		self.cspeed = 0.

	def set_maps(self):
		super().set_maps()

		self.diff_num_flux_map.update({
			base_diff_num_flux_type.SIP : 
				base_fcns.SIP,
			})

	class StateVariables(Enum):
		Scalar = "u"

	class AdditionalVariables(Enum):
	    MaxWaveSpeed = "\\lambda"

	def get_conv_flux_interior(self, Uq):
		c = self.c
		F = np.expand_dims(c*Uq, axis=-1)

		return F, None

	def get_diff_flux_interior(self, Uq, gUq):
		al = self.al
		F = al * gUq

		return F

	def compute_additional_variable(self, var_name, Uq, flag_non_physical):
		sname = self.AdditionalVariables[var_name].name

		if sname is self.AdditionalVariables["MaxWaveSpeed"].name:
			# Max wave speed is the advection speed
			scalar = np.full([Uq.shape[0], 1, 1], self.cspeed)
		else:
			raise NotImplementedError

		return scalar

In [ ]:
# SIP
# Geometric and order constants
hL = self.hL
hR = self.hR
eta = self.eta

# Jump (L - R)
dU = UqL - UqR

# Normalize the normal vectors
n_mag = np.linalg.norm(normals, axis=2, keepdims=True)
n_hat = normals/n_mag
# Pad normal magnitudes to (ne, nq, 1, 1)
n_mag_padded = np.expand_dims(normals, axis=-1)

# Tensor product of normal vector with jump
dUxn = np.einsum('ijk, ijl -> ijlk', n_hat, dU)

def get_diff_flux_interior(self, Uq, gUq):
  # State-dependent diffusion: flux k(U) * grad U is linear in gUq
  return 1.0 * gUq

# Jump flux w.r.t. left state: k(U_L) [[ U ]]
Fv_dir_jump_L = get_diff_flux_interior(UqL, dUxn)
# k(U_L) [[ U ]] * n /2
FvL = 0.5 * n_mag_padded * Fv_dir_jump_L

# Jump flux w.r.t. right state: k(U_R) [[ U ]]
Fv_dir_jump_R = get_diff_flux_interior(UqR, dUxn)
# k(U_R) [[ U ]] * n /2
FvR = 0.5 * n_mag_padded * Fv_dir_jump_R

Fv_dir = get_diff_flux_interior(UqL, gUqL) \
  - eta / hL * Fv_dir_jump_L \
  + get_diff_flux_interior(UqR, gUqR) \
  - eta / hL * Fv_dir_jump_R
# Projection of directional flux onto normal
Fv = np.einsum('ijl, ijkl -> ijk', normals, 0.5 * Fv_dir)


In [22]:
F = np.stack((solver.state_coeffs, solver.state_coeffs), axis=-1)
F.shape

(1215, 3, 8, 2)

In [28]:
elem_helpers.djac_elems[0,0,...]

array([3259.041164340054])

In [38]:
elem_helpers.quad_wts.shape

(9, 1)

In [39]:
elem_helpers.djac_elems.shape

(1215, 9, 1)

In [45]:
solver.order

1

In [44]:
F.shape, elem_helpers.quad_wts.shape, elem_helpers.djac_elems.shape

((1215, 3, 1, 2), (9, 1), (1215, 9, 1))

In [ ]:
# Interpolate state at quad points
Uq = helpers.evaluate_state(Uc, basis_val,
    skip_interp=self.basis.skip_interp) # [ne, nq, ns]
# Interpolate gradient of state at quad points
gUq = solver.evaluate_gradient(Uc, basis_phys_grad_elems)

In [51]:
F.shape

(1215, 9, 1, 2)

In [49]:
# solver_tools.calculate_volume_flux_integral(
# 					solver, solver.elem_helpers, F)



''' F extended to quadrature points -> volume integral Int(f phi' dx) residual
Compute volume flux integral: see solver/tools.py/calculate_volume_flux_integral
'''

# Synthetic flux at quadrature points
F = np.ones((*solver.elem_helpers.djac_elems.shape[0:2], 1, 2))

# Target computation:
res_elem = np.einsum('ijnl, ijkl, jm, ijm -> ink',
    solver.elem_helpers.basis_phys_grad_elems,
    F,
    solver.elem_helpers.quad_wts,
    solver.elem_helpers.djac_elems)

# Calculate residual
# res_elem = np.einsum('ijnl, ijkl -> ink', elem_helpers.basis_phys_grad_elems, F_quad)

In [19]:
solver.mesh
solver.elem_helpers.

In [ ]:
def fem2d_DG(coord, conn, nGauss,fFunc, eta, pipedMatrices):

    ne = conn.shape[0]
    A        = np.empty((3*ne, 3*ne))
    B_inner  = np.empty((3*ne, 3*ne))
    B_bdry   = np.empty((3*ne, 3*ne))
    S_bdry   = np.empty((3*ne, 3*ne))
    S_inner  = np.empty((3*ne, 3*ne))
    F = np.zeros((3*ne,1))

    # DG Assembly
    for k in range(ne):
      localCoord1 = coord(conn(k,:)',:);
      # Block assembly of body contributions
      A(3*k-2:3*k,3*k-2:3*k) = A(3*k-2:3*k,3*k-2:3*k) + ...
          localABody(localCoord1, nGauss);
      A[3*k:3*k+3, 3*k:3*k+3] += localABody(localCoord1, nGauss);
      # Assembly of forcing function
      F[3*k:3*k+3] = 0.0 #localF(localCoord1, nGauss, fFunc);
    # For each edge
    for i = 1:length(e)
      # Get connected elements
      connectedElements = e2t(i,:);
      # Get coordinates of the vertices of the edge
      edgeCoords = coord(e(i,:),:);
      # Check if this edge is on the domain boundary
      if connectedElements(2) == 0:
        # Get indices of sparse matrix to write to
        destination = 3*connectedElements(1)-2:3*connectedElements(1);
        # Grab coordinates of the element
        localCoords = coord(conn(connectedElements(1),:)',:);
        # Assemble jump-average contribution to B (boundary version)
        B_bdry[destination, destination] += \
            localBBdry(localCoords, edgeCoords, nGauss )
        # Assemble penalty contribution to S (boundary version)
        S_bdry[destination, destination] += \
            localSBdry(localCoords, edgeCoords, nGauss )
      else:
        # Get indices of sparse matrix to write to
        destination = [...
            (3*connectedElements(1)-2:3*connectedElements(1)), ...
            (3*connectedElements(2)-2:3*connectedElements(2))]';
        # Grab coordinates of both elements
        localCoord1 = coord(conn(connectedElements(1),:)',:);
        localCoord2 = coord(conn(connectedElements(2),:)',:);
        # Assemble jump-average contribution to B (interior version)
        B_inner[destination, destination] += \
            localBFace( localCoord1, localCoord2, nGauss );
        # Assemble penalty contribution to S (interior version)
        S_inner[destination, destination] += \
            localSFace( localCoord1, localCoord2, nGauss );
    # Output matrices to pipeline
    output_mats = {"A": A,
      "B_inner": B_inner,
      "B_bdry": B_bdry,
      "S_inner": S_inner,
      "S_bdry": S_bdry,
      "F": F,
      "LHS": A - B_inner - B_bdry - B_inner.T - B_bdry.T +
        eta*S_inner + eta*S_bdry
    }
    np.linalg.solve(output_mats["LHS"], F)

def localF( localCoord, nGauss, fFunc ):
  # Return int(f * phi)
    # Construct local f vector using gauss quadrature
    # Build affine map
    M_K = [(localCoord(2,:) - localCoord(1,:)) ; ...
        (localCoord(3,:) - localCoord(1,:))]';
    T_K = @(refCoord) M_K * refCoord + localCoord(1,:)';
    # Compute determinant
    detM_K = abs(det(M_K));
    # Define global basis functions in reference element
    phi_hat = {@(refCoord) 1 - refCoord(1) - refCoord(2);
               @(refCoord) refCoord(1);
               @(refCoord) refCoord(2)};
    # Construct local f vector
    Floc = zeros(3,1);
    for i in range(3):
      integrand = @(xi) fFunc(T_K(xi)) * phi_hat{i}(xi);
      Floc(i) = detM_K * gauss_quad2d(integrand, nGauss);

# Local stiffness matrix
def localABody( localCoord, nGauss)
    % Construct local matrix using gauss quadrature
    % Build affine map
    M_K = [(localCoord(2,:) - localCoord(1,:)) ; ...
        (localCoord(3,:) - localCoord(1,:))]';
    % Cache inverse map
    M_K_inv = inv(M_K);
    % Compute determinant
    detM_K = abs(det(M_K));
    % Define derivatives global basis functions in reference element
    grad_phi_hat = {@(refCoord) [-1; -1];
                    @(refCoord) [1; 0];
                    @(refCoord) [0; 1]};
    % Construct local A matrix (for triangular mesh)
    Aloc = zeros(3,3);
    for i in range(3):
      for j in range(3):
        integrand = @(xi) grad_phi_hat{i}(xi)' * ...
            M_K_inv * M_K_inv' * grad_phi_hat{j}(xi);
        Aloc(i,j) = gauss_quad2d(integrand, nGauss);
        Aloc(i,j) = Aloc(i,j) * detM_K;
  return Aloc

# Local 6x6 matrix B for interior faces
def localBFace( localCoord1, localCoord2, nGauss )
    % Construct local face contributions using gauss quadrature on a line
    
    % Identify overlapping coord pair
    edgeVertexCoords = localCoord1(...
        ismember(localCoord1(:,1), localCoord2(:,1)),:);
    % Re-map indices, so that vertices 1, 2 in each reference element are
    % the same
    [~, indices1] = ismember(edgeVertexCoords(:,:),localCoord1,'rows');
    remainder = setdiff(1:3, indices1);
    indexMap1 = [indices1; remainder];
    [~, indices2] = ismember(edgeVertexCoords(:,:),localCoord2,'rows');
    remainder = setdiff(1:3, indices2);
    indexMap2 = [indices2; remainder];
    
    % Get normal vector in original coordinates
    normalVector = [0, 1; -1, 0] * ...
        (edgeVertexCoords(1,:)-edgeVertexCoords(2,:))';
    % Flip normal vector if pointing inside the element 1
    if normalVector' * (localCoord1(indexMap1(3),:) - ...
            localCoord1(indexMap1(1),:))' > 0
        normalVector = -normalVector;
    end
    % Normalize
    normalVector = normalVector / norm(normalVector);
    
    % Build affine transformations
    M_K_1 = ...
        [(localCoord1(indexMap1(2),:) - localCoord1(indexMap1(1),:)) ; ...
        (localCoord1(indexMap1(3),:) - localCoord1(indexMap1(1),:))]';
    % Inverse of 3x3 matrix
    M_K_1_inv_trn = inv(M_K_1)';
    M_K_2 = ...
        [(localCoord2(indexMap2(2),:) - localCoord2(indexMap2(1),:)) ; ...
        (localCoord2(indexMap2(3),:) - localCoord2(indexMap2(1),:))]';
    M_K_2_inv_trn = inv(M_K_2)';
    
    % Edge map
    E_K_hat = @(s) [0.5; 0] + 0.5 * s * [1; 0];
    % Compute line length of the shared edge
    lineLength = norm(edgeVertexCoords(1,:)' - ...
        edgeVertexCoords(2,:)');
    % Define global basis functions in reference element
    phi_hat = {@(refCoord) 1 - refCoord(1) - refCoord(2);
               @(refCoord) refCoord(1);
               @(refCoord) refCoord(2)};
    % Define derivatives of global basis functions in reference element
    grad_phi_hat = {@(refCoord) [-1; -1];
                    @(refCoord) [1; 0];
                    @(refCoord) [0; 1]};
    % Construct local B11 matrix
    Bloc11 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            integrand = @(s) normalVector' * 0.5 * (...
                M_K_1_inv_trn * grad_phi_hat{j}(E_K_hat(s)) * ...
                phi_hat{i}(E_K_hat(s)));
              Bloc11(indexMap1(i),indexMap1(j)) = 0.5 * lineLength * ...
                gauss_quad1d(integrand, nGauss);
        end
    end
    % Construct local B12 matrix
    Bloc12 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            integrand = @(s) normalVector' * 0.5 * (...
                M_K_2_inv_trn * grad_phi_hat{j}(E_K_hat(s)) * ...
                phi_hat{i}(E_K_hat(s)));
            Bloc12(indexMap1(i),indexMap2(j)) = 0.5 * lineLength * ...
                gauss_quad1d(integrand, nGauss);
        end
    end
    % Construct local B21 matrix
    Bloc21 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            integrand = @(s) normalVector' * 0.5 * (...
                M_K_1_inv_trn * grad_phi_hat{j}(E_K_hat(s)) * ...
                phi_hat{i}(E_K_hat(s)));
            Bloc21(indexMap2(i),indexMap1(j)) = 0.5 * lineLength * ...
                gauss_quad1d(integrand, nGauss);
        end
    end
    % Construct local B22 matrix
    Bloc22 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            integrand = @(s) normalVector' * 0.5 * (...
                M_K_2_inv_trn * grad_phi_hat{j}(E_K_hat(s)) * ...
                phi_hat{i}(E_K_hat(s)));
              Bloc22(indexMap2(i),indexMap2(j)) = 0.5 * lineLength * ...
                gauss_quad1d(integrand, nGauss);
        end
    end
    % "Subassembly" of 6x6 local B matrix
    Bloc = [Bloc11, Bloc12; ...
        -Bloc21, -Bloc22];
    return Bloc
end

% Local 6x6 matrix S for interior faces
function Sloc = localSFace( localCoord1, localCoord2, nGauss )    
    % Identify overlapping coord pair
    edgeVertexCoords = localCoord1(...
        ismember(localCoord1(:,1), localCoord2(:,1)),:);
    % Re-map indices, so that vertices 1, 2 in each reference element are
    % the same
    [~, indices1] = ismember(edgeVertexCoords(:,:),localCoord1,'rows');
    remainder = setdiff(1:3, indices1);
    indexMap1 = [indices1; remainder];
    [~, indices2] = ismember(edgeVertexCoords(:,:),localCoord2,'rows');
    remainder = setdiff(1:3, indices2);
    indexMap2 = [indices2; remainder];
    % Edge map
    E_K_hat = @(s) [0.5; 0] + 0.5 * s * [1; 0];
    % Define global basis functions in reference element
    phi_hat = {@(refCoord) 1 - refCoord(1) - refCoord(2);
               @(refCoord) refCoord(1);
               @(refCoord) refCoord(2)}; 
    % Construct local S11 matrix
    Sloc11 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            penalty_integrand = @(s) ...
                phi_hat{i}(E_K_hat(s)) * ...
                phi_hat{j}(E_K_hat(s));
            Sloc11(indexMap1(i),indexMap1(j)) = ...
                0.5 * gauss_quad1d(penalty_integrand, nGauss);
        end
    end
    % Construct local S12 matrix
    Sloc12 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            penalty_integrand = @(s) ...
                phi_hat{i}(E_K_hat(s)) * ...
                phi_hat{j}(E_K_hat(s));
            Sloc12(indexMap1(i),indexMap2(j)) = ...
                0.5 * gauss_quad1d(penalty_integrand, nGauss);
        end
    end
    % Construct local S21 matrix
    Sloc21 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            penalty_integrand = @(s) ...
                phi_hat{i}(E_K_hat(s)) * ...
                phi_hat{j}(E_K_hat(s));
            Sloc21(indexMap2(i),indexMap1(j)) = ...
                0.5 * gauss_quad1d(penalty_integrand, nGauss);
        end
    end
    % Construct local S22 matrix
    Sloc22 = zeros(3,3);
    for i = 1:3
        for j = 1:3
            penalty_integrand = @(s) ...
                phi_hat{i}(E_K_hat(s)) * ...
                phi_hat{j}(E_K_hat(s));
            Sloc22(indexMap2(i),indexMap2(j)) = ...
                0.5 * gauss_quad1d(penalty_integrand, nGauss);
        end
    end
    % "Subassembly" of 6x6 matrix
    Sloc = [Sloc11, -Sloc12; ...
        -Sloc21, Sloc22];
end

% Local 3x3 matrix S for boundary faces
function Sloc = localSBdry( localCoord, edgeCoords, nGauss )
    % Re-map indices, so that vertices 1, 2 in reference element lie on the
    % boundary edge
    [~, indices] = ismember(edgeCoords(:,:),localCoord,'rows');
    remainder = setdiff(1:3, indices);
    indexMap = [indices; remainder];
    % Edge map
    E_K_hat = @(s) [0.5; 0] + 0.5 * s * [1; 0];
    % Define global basis functions in reference element
    phi_hat = {@(refCoord) 1 - refCoord(1) - refCoord(2);
               @(refCoord) refCoord(1);
               @(refCoord) refCoord(2)};
    % Initialize local matrix
    Sloc = zeros(3,3);
    for i = 1:3
        for j = 1:3
            penalty_integrand = @(s) ...
                phi_hat{i}(E_K_hat(s)) * ...
                phi_hat{j}(E_K_hat(s));
            Sloc(indexMap(i),indexMap(j)) = ...
                0.5 * gauss_quad1d(penalty_integrand, nGauss);
        end
    end
end

% Local 3x3 matrix B for boundary faces
function Bloc = localBBdry( localCoord, edgeCoords, nGauss )
    % Re-map indices, so that vertices 1, 2 in reference element lie on the
    % boundary edge
    [~, indices] = ismember(edgeCoords(:,:),localCoord,'rows');
    remainder = setdiff(1:3, indices);
    indexMap = [indices; remainder];
    % Get normal vector in original coordinates
    normalVector = [0, 1; -1, 0] * ...
        (edgeCoords(1,:)-edgeCoords(2,:))';
    % Flip normal vector if pointing inside the element 1
    if normalVector' * (localCoord(indexMap(3),:) - ...
            localCoord(indexMap(1),:))' > 0
        normalVector = -normalVector;
    end
    % Normalize
    normalVector = normalVector / norm(normalVector);
    % Build affine transformations
    M_K = ...
        [(localCoord(indexMap(2),:) - localCoord(indexMap(1),:)) ; ...
        (localCoord(indexMap(3),:) - localCoord(indexMap(1),:))]';
    % Inverse transpose of a 3x3
    M_K_inv_trn = inv(M_K)';
    % Edge map
    E_K_hat = @(s) [0.5; 0] + 0.5 * s * [1; 0];
    % Compute line length of the shared edge
    lineLength = norm(edgeCoords(1,:)' - ...
        edgeCoords(2,:)');
    % Define global basis functions in reference element
    phi_hat = {@(refCoord) 1 - refCoord(1) - refCoord(2);
               @(refCoord) refCoord(1);
               @(refCoord) refCoord(2)};
    % Define derivatives of global basis functions in reference element
    grad_phi_hat = {@(refCoord) [-1; -1];
                    @(refCoord) [1; 0];
                    @(refCoord) [0; 1]};
    % Construct local A_face matrix
    Bloc = zeros(3,3);
    for i = 1:3
        for j = 1:3
            integrand = @(s) normalVector' * (...
                M_K_inv_trn * (grad_phi_hat{j}(E_K_hat(s)) * ...
                phi_hat{i}(E_K_hat(s))));
            Bloc(indexMap(i),indexMap(j)) = ...
                0.5 * lineLength * gauss_quad1d(integrand, nGauss);
        end
    end    
end